In [1]:
import networkx as nx
import os
import pybel
from se_kge.graph_preprocessing import get_drugbank_graph, get_sider_graph, combine_pubchem_drugbank

# Building SIDER and DrugBank graphs

In [2]:
sider_graph = get_sider_graph()

indications: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153970/153970 [01:33<00:00, 1655.11it/s]


In [3]:
drugbank_graph = get_drugbank_graph()

Mapping drug-protein interactions to BEL: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 45856/45856 [02:38<00:00, 289.52it/s]


# Combining both graphs into a complete graph

In [4]:
drugbank_pubchem_mapping = os.path.join(os.pardir, "resources", "drugbank_pubchem_mapping.tsv")

In [5]:
full_graph = combine_pubchem_drugbank(drugbank_pubchem_mapping, drugbank_graph, sider_graph)

create pubchem-drugbank mapping dictionary: 6194it [00:03, 1852.80it/s]


Number of nodes that were not relabeled 1212


Removing nodes that were not relabeled: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1212/1212 [00:00<00:00, 4073.44it/s]


The number of nodes in the combined graph is 14292


In [6]:
sider_side_effects = 0
sider_drugs = 0
for node in sider_graph.nodes():
    if node.namespace == 'umls':
        sider_side_effects+=1
    else:
        sider_drugs+=1
print('SIDER graph has a total of %d nodes' % len(sider_graph.nodes()))
print('The number of Side Effects nodes is %d' % sider_side_effects)
print('The number of Drugs nodes is %d' % sider_drugs)

SIDER graph has a total of 8497 nodes
The number of Side Effects nodes is 6990
The number of Drugs nodes is 1507


In [7]:
drugbank_proteins = 0
drugbank_drugs = 0
for node in drugbank_graph.nodes():
    if node.namespace == 'uniprot':
        drugbank_proteins+=1
    else:
        drugbank_drugs+=1
print('DrugBank Graph has a total of %d nodes' % len(drugbank_graph.nodes()))
print('The number of Proteins nodes is %d' % drugbank_proteins)
print('The number of drugs nodes is %d' % drugbank_drugs)

DrugBank Graph has a total of 7517 nodes
The number of Proteins nodes is 2559
The number of drugs nodes is 4958


In [8]:
proteins = 0
side_effects = 0
drugs = 0

for node in full_graph.nodes():
    if node.namespace == 'uniprot':
        proteins+=1
    elif node.namespace == 'umls':
        side_effects+=1
    else:
        drugs+=1

print('The combined graph has a total of %d nodes' % len(full_graph.nodes()))
print('The number of Proteins nodes is %d' % proteins)
print('The number of Side Effects nodes is %d' % side_effects)
print('The number of Drugs nodes is %d' % drugs)

The combined graph has a total of 14292 nodes
The number of Proteins nodes is 2559
The number of Side Effects nodes is 6990
The number of Drugs nodes is 4743


In [9]:
nx.write_edgelist(sider_graph, os.path.join(os.pardir, "resources", "sider_graph.edgelist"))
pybel.to_pickle(sider_graph, os.path.join(os.pardir, "resources", "sider_graph.pickle"))

In [10]:
nx.write_edgelist(drugbank_graph, os.path.join(os.pardir, "resources", "drugbank_graph.edgelist"))
pybel.to_pickle(drugbank_graph, os.path.join(os.pardir, "resources", "drugbank_graph.pickle"))

In [11]:
nx.write_edgelist(full_graph, os.path.join(os.pardir, "resources", "fullgraph_without_sim.edgelist"))
pybel.to_pickle(full_graph, os.path.join(os.pardir, "resources", "fullgraph_without_sim.pickle"))

# combining the full graph with chemical similarity graph
P.S: Calculating chemical similarities and creating the graph is done in a different notebook because it needs the RDkit package and environment

In [29]:
chem_sim_graph = pybel.from_pickle(os.path.join(os.pardir, "resources", "chem_sim_graph.pickle"))

In [30]:
len(chem_sim_graph.nodes())

1488